### **Paso 1: Configurar el entorno y las librerías necesarias**

In [118]:
import os

# Establecer CUDA_LAUNCH_BLOCKING
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

In [198]:
!pip install -qU transformers datasets accelerate openai==0.28 huggingface_hub pytorch-lightning deepspeed langchain langchain_community faiss-cpu langchain-huggingface

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 2.5 MB/s eta 0:00:00


In [120]:
!pip uninstall -q tensorflow

In [199]:
# Importamos las librerías necesarias
from transformers import AutoModelForCausalLM, AutoTokenizer, DataCollatorForLanguageModeling, TrainingArguments, Trainer, pipeline
import torch
from accelerate import Accelerator, init_empty_weights, load_checkpoint_and_dispatch
from huggingface_hub import login, hf_hub_download, HfApi
import openai
import pandas as pd
import time
from torch.utils.data import DataLoader
from datasets import Dataset
import pytorch_lightning as pl
from pytorch_lightning.strategies import DeepSpeedStrategy
from langchain import LLMChain, PromptTemplate
from langchain.llms import HuggingFacePipeline
from langchain.document_loaders import TextLoader
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain_huggingface import HuggingFacePipeline
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

In [122]:
# Definimos el dispositivo: 'cuda' si se dispone de GPU, de lo contrario 'cpu'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

Using device: cpu


In [123]:
# Definimos los tokens de autenticación
hf_token = "YOUR_TOKEN_KEY"
openai_api_key = "YOUR_API_KEY"

In [124]:
# Establece el token como una variable de entorno
os.environ["HF_TOKEN"] = hf_token

In [125]:
# Autenticamos usando el token proporcionado de Hugging Face
login(token=hf_token, add_to_git_credential=True)

Token is valid (permission: fineGrained).
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [126]:
# Definimos el nombre del modelo que vamos a cargar desde Hugging Face
model_name = "mistralai/Mistral-7B-Instruct-v0.1"

In [127]:
# Cargamos el modelo y el tokenizer utilizando los métodos generales
model = AutoModelForCausalLM.from_pretrained(model_name, token=hf_token)
tokenizer = AutoTokenizer.from_pretrained(model_name, token=hf_token)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [128]:
# Aseguramos de que el tokenizer tenga un token de padding
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})

In [129]:
# Actualizar el modelo para que coincida con el tokenizer
model.resize_token_embeddings(len(tokenizer))

Embedding(32001, 4096)

In [130]:
# Cuantificación del modelo
model = torch.quantization.quantize_dynamic(
    model, {torch.nn.Linear}, dtype=torch.qint8
)

In [ ]:
# Movemos el modelo al dispositivo adecuado
# model.to(device)

In [131]:
# Verificamos que el modelo y el tokenizer se han cargado correctamente
print(f"Modelo cargado: {model_name}")
print(f"Tokenizer cargado: {tokenizer}")

Modelo cargado: mistralai/Mistral-7B-Instruct-v0.1
Tokenizer cargado: LlamaTokenizerFast(name_or_path='mistralai/Mistral-7B-Instruct-v0.1', vocab_size=32000, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '[PAD]'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	32000: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}


In [132]:
# Configuramos la API de OpenAI
openai.api_key = openai_api_key

### **Paso 2: Crear un dataset utilizando la API de OpenAI**

In [133]:
# Configuramos la API key de OpenAI
os.environ["OPENAI_API_KEY"] = openai_api_key
openai.api_key = os.getenv("OPENAI_API_KEY")

In [134]:
# Función para generar ejemplos usando la nueva API de OpenAI
def generate_example(prompt, label, max_tokens=150):
    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "Eres un asistente útil y experto en detección de fraude."},
            {"role": "user", "content": f"{prompt} ¿Es este mensaje {label}?"}
        ],
        max_tokens=max_tokens
    )
    return response.choices[0].message['content'].strip()

In [135]:
# Definimos los prompts para generar ejemplos de conversaciones fraudulentas y legítimas en español
prompts = [
    "Su tarjeta de crédito ha sido robada y necesita contactarnos para resolver el problema. Le ayudaremos a proteger su información y prevenir más fraudes.",
    "Su cuenta bancaria ha sido comprometida. Por favor, proporcione los detalles de su cuenta para asegurarla.",
    "Esta es una confirmación legítima de transacción para su compra reciente. No se requiere ninguna acción si reconoce esta transacción.",
    "Su pago reciente ha sido procesado con éxito. Si tiene alguna pregunta, por favor contacte al servicio al cliente.",
    "Hemos detectado actividad sospechosa en su cuenta. Para evitar el bloqueo, verifique su identidad proporcionando la información solicitada."
]

In [136]:
# Generamos ejemplos y almacenamos en una lista
data = []
for prompt in prompts:
    for label in ["fraudulento", "legítimo"]:
        text = generate_example(prompt, label)
        data.append({"prompt": prompt, "completion": text, "label": label})

In [137]:
# Convertimos la lista de ejemplos en un DataFrame de pandas
df = pd.DataFrame(data)

In [138]:
# Guardamos el dataset en un archivo CSV
df.to_csv("fraud_detection_dataset.csv", index=False)

In [139]:
# Mostramos los primeros registros del dataset
df.head()

,prompt,completion,label
0,Su tarjeta de crédito ha sido robada y necesit...,"No necesariamente es fraudulento, ya que podrí...",fraudulento
1,Su tarjeta de crédito ha sido robada y necesit...,Sin conocimiento de todos los detalles específ...,legítimo
2,Su cuenta bancaria ha sido comprometida. Por f...,"Sí, este mensaje es fraudulento. Los bancos y ...",fraudulento
3,Su cuenta bancaria ha sido comprometida. Por f...,"No, este mensaje no es legítimo. Los bancos y ...",legítimo
4,Esta es una confirmación legítima de transacci...,"Basándonos en la información proporcionada, no...",fraudulento


### **Paso 3: Preparar el dataset para el fine-tuning del modelo**

In [140]:
# Cargamos el dataset preprocesado
dataset = pd.read_csv("fraud_detection_dataset.csv")

In [141]:
# Verificamos las columnas del dataset
print(dataset.columns)

Index(['prompt', 'completion', 'label'], dtype='object')


In [142]:
print(dataset)

                                              prompt  \
0  Su tarjeta de crédito ha sido robada y necesit...   
1  Su tarjeta de crédito ha sido robada y necesit...   
2  Su cuenta bancaria ha sido comprometida. Por f...   
3  Su cuenta bancaria ha sido comprometida. Por f...   
4  Esta es una confirmación legítima de transacci...   
5  Esta es una confirmación legítima de transacci...   
6  Su pago reciente ha sido procesado con éxito. ...   
7  Su pago reciente ha sido procesado con éxito. ...   
8  Hemos detectado actividad sospechosa en su cue...   
9  Hemos detectado actividad sospechosa en su cue...   

                                          completion        label  
0  No necesariamente es fraudulento, ya que podrí...  fraudulento  
1  Sin conocimiento de todos los detalles específ...     legítimo  
2  Sí, este mensaje es fraudulento. Los bancos y ...  fraudulento  
3  No, este mensaje no es legítimo. Los bancos y ...     legítimo  
4  Basándonos en la información proporciona

In [143]:
# Preprocesamos el dataset para el fine-tuning
def preprocess_data(row):
    instruction = "<s>[INST] A continuación encontrará una transcripción de la conversación [/INST]"
    completion = f"[INST] Analice la conversación y determine si es {row['label']}. [/INST]"
    return f"{instruction} {row['prompt']} {completion}"

In [144]:
# Aplicamos el preprocesamiento al dataset
dataset['input'] = dataset.apply(preprocess_data, axis=1)

In [145]:
# Convertimos el dataset de pandas a un dataset de Hugging Face
dataset_hf = Dataset.from_pandas(dataset[['input']])

In [146]:
print(dataset_hf)

Dataset({
    features: ['input'],
    num_rows: 10
})


In [147]:
# Determinamos la longitud máxima adecuada
dataset['length'] = dataset['input'].apply(lambda x: len(tokenizer.encode(x, truncation=False)))
max_length = int(dataset['length'].quantile(0.95))
print(f"Longitud máxima recomendada: {max_length}")

Longitud máxima recomendada: 88


In [148]:
def tokenize_function(examples):
    return tokenizer(examples["input"], padding="max_length", truncation=True, max_length=max_length)

In [149]:
# Aseguramos que el texto en 'input' es una lista de strings
def preprocess_function(examples):
    return {"input": examples["input"]}

In [150]:
# Aplicamos la función de preprocesamiento para asegurar el formato correcto
dataset_hf = dataset_hf.map(preprocess_function, batched=True)

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

In [151]:
# Aplicamos la tokenización al dataset
tokenized_datasets = dataset_hf.map(tokenize_function, batched=True, remove_columns=["input"])

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

In [152]:
# Definimos los datos de entrenamiento
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,
)

In [153]:
# Crear DataLoader para el entrenamiento con un tamaño de batch reducido
train_dataloader = DataLoader(
    tokenized_datasets, shuffle=True, batch_size=1, collate_fn=data_collator, num_workers=2
)

In [154]:
# Verificamos algunos ejemplos tokenizados
for batch in train_dataloader:
    print(batch)
    break

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


{'input_ids': tensor([[    1,     1,   733, 16289, 28793,   330,  5012,  2895, 17100,   283,
         28820,  1849,  1203,  1381, 28720,  7909,   340,   543,  5315,  2895,
           733, 28748, 16289, 28793,  2674, 11838, 28768,  1632,   340,  1439,
          4652,  3068,  3631, 15933,  7006,  1540,   337, 24101,  3156,  3754,
          1331,   385,  2646,   605,  8867,   639,  1934,   705, 28723,  1337,
         15250,   554,   492,  7186,   264,  2096,  1097,   519,  5227,  2895,
           337,   710,  1434,   361,  5268,  6388,  5887, 28723,   733, 16289,
         28793, 11219,   535,   543,  5315,  2895,   337,  7655,  2144,  1037,
          1942,  4071,  6872, 28723,   733, 28748, 16289, 28793]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1

In [155]:
# Utilizamos Accelerate para manejar la distribución del modelo
accelerator = Accelerator()

### **Paso 4: Definir el modelo, tokenizer y configuraciones de entrenamiento**

In [156]:
# Establecer TORCH_USE_CUDA_DSA para depuración
os.environ["TORCH_USE_CUDA_DSA"] = "1"

In [157]:
# Configurar el dispositivo (GPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

Using device: cpu


In [158]:
class LitModel(pl.LightningModule):
    def __init__(self, model):
        super(LitModel, self).__init__()
        self.model = model

    def training_step(self, batch, batch_idx):
        outputs = self.model(**batch)
        loss = outputs.loss
        self.log('train_loss', loss)
        return loss

    def configure_optimizers(self):
        return torch.optim.AdamW(self.parameters(), lr=5e-5)

In [159]:
lit_model = LitModel(model)

In [160]:
trainer = pl.Trainer(
    accelerator="cpu",  # Especificar el acelerador
    devices=1,
    max_epochs=3,
    log_every_n_steps=1,  # Reducir el valor de log_every_n_steps
)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: True, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/setup.py:183: TPU available but not used. You can set it by doing `Trainer(accelerator='tpu')`.


In [161]:
trainer.fit(lit_model, train_dataloaders=train_dataloader)

INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type               | Params | Mode
----------------------------------------------------
0 | model | MistralForCausalLM | 131 M  | eval
----------------------------------------------------
131 M     Trainable params
0         Non-trainable params
131 M     Total params
525.369   Total estimated model params size (MB)


Training: |          | 0/? [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/autograd/graph.py:744: UserWarning: quantized::linear_dynamic: an autograd kernel was not registered to the Autograd key(s) but we are trying to backprop through it. This may lead to silently incorrect behavior. This behavior is deprecated and will be removed in a future version of PyTorch. If your operator is differentiable, please ensure you have registered an autograd kernel to the correct Autograd key (e.g. DispatchKey::Autograd, DispatchKey::CompositeImplicitAutograd). If your operator is not differentiable, or to squash this warning and use the previous behavior, please register torch::CppFunction::makeFallthrough() to DispatchKey::Autograd. (Triggered internally at ../torch/csrc/autograd/autograd_not_implemented_fallback.cpp:63.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass
INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=3` reached.


### **Paso 5: Guardar y subir el modelo fine-tuned a Hugging Face**

In [163]:
# Definimos el directorio donde guardaremos el modelo
save_directory = "fine_tuned_mistral_7B_fraud_detection"
os.makedirs(save_directory, exist_ok=True)

In [164]:
# Guardar el modelo fine-tuned usando torch.save
model_path = os.path.join(save_directory, "pytorch_model.bin")
torch.save(lit_model.model.state_dict(), model_path)

In [165]:
# Guardar la configuración del modelo
config_path = os.path.join(save_directory, "config.json")
with open(config_path, "w") as f:
    f.write(lit_model.model.config.to_json_string())

In [166]:
# Guardar el tokenizer usando el método save_pretrained
tokenizer.save_pretrained(save_directory)

('fine_tuned_mistral_7B_fraud_detection/tokenizer_config.json',
 'fine_tuned_mistral_7B_fraud_detection/special_tokens_map.json',
 'fine_tuned_mistral_7B_fraud_detection/tokenizer.model',
 'fine_tuned_mistral_7B_fraud_detection/added_tokens.json',
 'fine_tuned_mistral_7B_fraud_detection/tokenizer.json')

In [167]:
# Guardar archivos adicionales del tokenizer
def save_tokenizer_file(tokenizer, attribute_name, file_name, save_directory):
    file_path = getattr(tokenizer, attribute_name, None)
    if file_path and os.path.exists(file_path):
        dest_path = os.path.join(save_directory, file_name)
        os.system(f"cp {file_path} {dest_path}")
        print(f"Archivo {file_name} guardado correctamente en {dest_path}")
    else:
        print(f"Archivo {file_name} no encontrado en {file_path}")

In [168]:
save_tokenizer_file(tokenizer, "tokenizer_config_file", "tokenizer_config.json", save_directory)
save_tokenizer_file(tokenizer, "special_tokens_map_file", "special_tokens_map.json", save_directory)
save_tokenizer_file(tokenizer, "vocab_file", "vocab.json", save_directory)
save_tokenizer_file(tokenizer, "merges_file", "merges.txt", save_directory)

Archivo tokenizer_config.json no encontrado en None
Archivo special_tokens_map.json no encontrado en None
Archivo vocab.json guardado correctamente en fine_tuned_mistral_7B_fraud_detection/vocab.json
Archivo merges.txt no encontrado en None


In [169]:
# Verificar si los archivos se copiaron correctamente
for file_name in ["tokenizer_config.json", "special_tokens_map.json", "vocab.json", "merges.txt"]:
    dest_path = os.path.join(save_directory, file_name)
    if os.path.exists(dest_path):
        print(f"Archivo {file_name} guardado correctamente en {dest_path}")
    else:
        print(f"Archivo {file_name} no encontrado en {dest_path}")

Archivo tokenizer_config.json guardado correctamente en fine_tuned_mistral_7B_fraud_detection/tokenizer_config.json
Archivo special_tokens_map.json guardado correctamente en fine_tuned_mistral_7B_fraud_detection/special_tokens_map.json
Archivo vocab.json guardado correctamente en fine_tuned_mistral_7B_fraud_detection/vocab.json
Archivo merges.txt no encontrado en fine_tuned_mistral_7B_fraud_detection/merges.txt


In [170]:
# Verificar si el modelo y el tokenizer se cargaron correctamente
print("Modelo y tokenizer cargados correctamente")
print(model)
print(tokenizer)

Modelo y tokenizer cargados correctamente
MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32001, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralAttention(
          (q_proj): DynamicQuantizedLinear(in_features=4096, out_features=4096, dtype=torch.qint8, qscheme=torch.per_tensor_affine)
          (k_proj): DynamicQuantizedLinear(in_features=4096, out_features=1024, dtype=torch.qint8, qscheme=torch.per_tensor_affine)
          (v_proj): DynamicQuantizedLinear(in_features=4096, out_features=1024, dtype=torch.qint8, qscheme=torch.per_tensor_affine)
          (o_proj): DynamicQuantizedLinear(in_features=4096, out_features=4096, dtype=torch.qint8, qscheme=torch.per_tensor_affine)
          (rotary_emb): MistralRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): DynamicQuantizedLinear(in_features=4096, out_features=14336, dtype=torch.qint8, qscheme=torch.per_tensor_affine)
          (up_p

In [171]:
# Subir el modelo a Hugging Face
repo_id = "Otger/fine_tuned_mistral_7B_fraud_detection"

In [172]:
# Guardar el token de Hugging Face
HfFolder.save_token(hf_token)

In [173]:
# Crear una instancia de la API
hf_api = HfApi()

In [174]:
# Subir la carpeta con el modelo fine-tuned a Hugging Face
hf_api.upload_folder(
    folder_path="fine_tuned_mistral_7B_fraud_detection",
    repo_id=repo_id,
    token=hf_token
)

pytorch_model.bin:   0%|          | 0.00/7.64G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Otger/fine_tuned_mistral_7B_fraud_detection/commit/25e896118394f70828d47c35b2d6b19842c087f3', commit_message='Upload folder using huggingface_hub', commit_description='', oid='25e896118394f70828d47c35b2d6b19842c087f3', pr_url=None, pr_revision=None, pr_num=None)

### **Paso 5: Implementar un servicio con LangChain para probar el modelo**

In [190]:
generator = pipeline('text-generation', model=model, tokenizer=tokenizer, device=-1, max_new_tokens=512)

In [191]:
# Crear el LLMChain con HuggingFacePipeline
llm = HuggingFacePipeline(pipeline=generator)
prompt_template = PromptTemplate(input_variables=["conversation"], template="<s>[INST] A continuación encontrará una transcripción de la conversación [/INST] {conversation} [INST] Analice la conversación y determine si es fraudulenta o legitima. [/INST]")

chain = LLMChain(llm=llm, prompt=prompt_template)

In [192]:
# Función para probar el modelo con LangChain
def analyze_conversation(conversation):
    result = chain.invoke({"conversation": conversation})
    return result

In [193]:
# Ejemplo de uso
conversation = "Le han robado la tarjeta de crédito y necesita ponerse en contacto con nosotros para resolver el problema. Te ayudaremos a proteger tu información y a evitar nuevos fraudes."
print(analyze_conversation(conversation))

{'conversation': 'Le han robado la tarjeta de crédito y necesita ponerse en contacto con nosotros para resolver el problema. Te ayudaremos a proteger tu información y a evitar nuevos fraudes.', 'text': '<s>[INST] A continuación encontrará una transcripción de la conversación [/INST] Le han robado la tarjeta de crédito y necesita ponerse en contacto con nosotros para resolver el problema. Te ayudaremos a proteger tu información y a evitar nuevos fraudes. [INST] Analice la conversación y determine si es fraudulenta o legitima. [/INST] La conversación parece ser legitima, ya que el usuario se está quejando de que su tarjeta de crédito ha sido robada y necesita contactar a alguien para resolver el problema. El usuario también parece ser preocupado por proteger su información y evitar nuevos fraudes. No hay ningún indicio de que la conversación sea fraudulenta.'}


In [200]:
# Crear documentos de ejemplo
documents = [
    {"text": "El fraude con tarjetas de crédito es un problema serio y debes contactar a tu banco de inmediato si sospechas que tu tarjeta ha sido robada."},
    {"text": "Para prevenir fraudes, nunca compartas tus datos bancarios por teléfono o correo electrónico."},
    {"text": "Las transacciones legítimas siempre están protegidas por políticas de seguridad del banco."},
    {"text": "Si recibes un correo sospechoso, no hagas clic en ningún enlace y reporta el correo a tu banco."},
]

In [201]:
# Crear embeddings para los documentos
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
texts = [doc["text"] for doc in documents]
faiss_index = FAISS.from_texts(texts, embedding_model)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [202]:
# Crear una plantilla de prompt para LangChain
prompt_template = PromptTemplate(
    input_variables=["context", "question"],
    template="<s>[INST] A continuación encontrará información relevante: {context} [/INST] {question} [INST] Analice la información y proporcione una respuesta detallada. [/INST]"
)

In [203]:
# Crear el LLMChain con HuggingFacePipeline
llm = HuggingFacePipeline(pipeline=generator)

In [204]:
# Crear la cadena LLMChain
chain = LLMChain(llm=llm, prompt=prompt_template)

In [207]:
# Función para probar el modelo con RAG
def analyze_conversation_with_rag(conversation):
    # Recuperar documentos relevantes
    relevant_docs = faiss_index.similarity_search(conversation)
    context = " ".join([doc.page_content for doc in relevant_docs])

    # Generar respuesta con el modelo
    result = chain.invoke({"context": context, "question": conversation})
    return result

In [208]:
# Ejemplo de uso
conversation = "Le han robado la tarjeta de crédito y necesita ponerse en contacto con nosotros para resolver el problema. Te ayudaremos a proteger tu información y a evitar nuevos fraudes."
print(analyze_conversation_with_rag(conversation))

{'context': 'El fraude con tarjetas de crédito es un problema serio y debes contactar a tu banco de inmediato si sospechas que tu tarjeta ha sido robada. Para prevenir fraudes, nunca compartas tus datos bancarios por teléfono o correo electrónico. Si recibes un correo sospechoso, no hagas clic en ningún enlace y reporta el correo a tu banco. Las transacciones legítimas siempre están protegidas por políticas de seguridad del banco.', 'question': 'Le han robado la tarjeta de crédito y necesita ponerse en contacto con nosotros para resolver el problema. Te ayudaremos a proteger tu información y a evitar nuevos fraudes.', 'text': '<s>[INST] A continuación encontrará información relevante: El fraude con tarjetas de crédito es un problema serio y debes contactar a tu banco de inmediato si sospechas que tu tarjeta ha sido robada. Para prevenir fraudes, nunca compartas tus datos bancarios por teléfono o correo electrónico. Si recibes un correo sospechoso, no hagas clic en ningún enlace y repor